In [1]:
import random
from copy import deepcopy

########### Low level functions ##########
## Get random name from list
def get_random_name(list):
    index = random.randrange(0, len(list))
    return list[index]

## Remove name from list
def remove_from_list(list, remove_obj):
    shortened_list = deepcopy(list)
    if remove_obj in shortened_list:
        shortened_list.remove(remove_obj)
    return shortened_list    
    
    
## Setup possible names
NAME_LIST = ["Tyler", "Nancy", "Jenny", "Sandy", "Kent", "Peter"]  # do not change
assignment_dict_1 = {}

## Setup possible recipient list
recipient_list = deepcopy(NAME_LIST)

## Assign first recipients
for i in range(0, len(NAME_LIST)):
    ## Get buyer
    buyer = NAME_LIST[i]
    
    ## Get random recipient, excluding buyer
    shortened_list = remove_from_list(recipient_list, buyer)
    recipient = get_random_name(shortened_list)
    
    ## Remove recipient from recipient list
    recipient_list.remove(recipient)
    
    ## Add buyer-> recipient pair
    assignment_dict_1[buyer] = [recipient]
    
## Check if worked
check_list = []
success = True
for key in assignment_dict_1:
    name_list = assignment_dict_1[key]
    if name_list in check_list:
        success = False
        print("ERROR. {0} is a double recipient".format(name_list))
    check_list.append(name_list)
print("Success? {0}".format(success))
print(assignment_dict_1)

print(assignment_dict_1["Tyler"][0])

print("*************************")

recipient_list = deepcopy(NAME_LIST)
## Assign second recipients
for i in range(0, len(NAME_LIST)):
    ## Get buyer
    buyer = NAME_LIST[i]
    
    ## Get random recipient, excluding buyer and their first recipient
    shortened_list = remove_from_list(recipient_list, buyer)
    shortened_list = remove_from_list(recipient_list, assignment_dict_1[buyer][0])
    recipient = get_random_name(shortened_list)
    
    ## Remove recipient from recipient list
    recipient_list.remove(recipient)
    
    ## Add buyer-> recipient pair
    assignment_dict_1[buyer].append(recipient)
    print("Ass {0}: {1}".format(i, assignment_dict_1))
    
    
## Check if worked
check_list = []
success = True
for key in assignment_dict_1:
    name_list = assignment_dict_1[key]
    if (name_list[0] is name_list[1]) or (name_list[0] is key) or (name_list[1] is key):
        success = False
        print("ERROR. {0} is a double recipient".format(name_list))
    check_list.append(name_list)
print("Success? {0}".format(success))
print(assignment_dict_1)

Success? True
{'Tyler': ['Jenny'], 'Nancy': ['Peter'], 'Jenny': ['Sandy'], 'Sandy': ['Tyler'], 'Kent': ['Nancy'], 'Peter': ['Kent']}
Jenny
*************************
Ass 0: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter'], 'Jenny': ['Sandy'], 'Sandy': ['Tyler'], 'Kent': ['Nancy'], 'Peter': ['Kent']}
Ass 1: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter', 'Nancy'], 'Jenny': ['Sandy'], 'Sandy': ['Tyler'], 'Kent': ['Nancy'], 'Peter': ['Kent']}
Ass 2: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter', 'Nancy'], 'Jenny': ['Sandy', 'Peter'], 'Sandy': ['Tyler'], 'Kent': ['Nancy'], 'Peter': ['Kent']}
Ass 3: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter', 'Nancy'], 'Jenny': ['Sandy', 'Peter'], 'Sandy': ['Tyler', 'Kent'], 'Kent': ['Nancy'], 'Peter': ['Kent']}
Ass 4: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter', 'Nancy'], 'Jenny': ['Sandy', 'Peter'], 'Sandy': ['Tyler', 'Kent'], 'Kent': ['Nancy', 'Sandy'], 'Peter': ['Kent']}
Ass 5: {'Tyler': ['Jenny', 'Tyler'], 'Nancy': ['Peter', 'Nancy'], 'Jenny

In [2]:
import random
from copy import deepcopy

########### Low level functions ##########
## Get random name from list
def get_random_name(list):
    index = random.randrange(0, len(list))
    return list[index]

## Remove name from list
def remove_name_from_list(list, remove_name):
    shortened_list = deepcopy(list)
    if remove_name in shortened_list:
        shortened_list.remove(remove_name)
    return shortened_list

## Remove list from list
def remove_list_from_list(list, remove_list):
    shortened_list = deepcopy(list)
    for i in range(0, len(remove_list)):
        shortened_list = remove_from_list(shortened_list, remove_list[i])
    return shortened_list

## Get random from list, excuding names
def get_random_name_excluding(list, exclude_list):
    return get_random_name(remove_list_from_list(list, exclude_list))

NAME_LIST = ["Tyler", "Nancy", "Jenny", "Sandy", "Kent", "Peter"]  # do not change

########### High level functions ##########
def get_name_assignment(full_name_list, assignment_dict):
    new_assignment_dict = deepcopy(assignment_dict)
    recipient_list = deepcopy(full_name_list)
    
    ## Assign recipients
    for i in range(0, len(full_name_list)):
        ## Get buyer
        buyer = full_name_list[i]

        ## Get random recipient, excluding buyer and repeat recipients  
        exclude_list = [buyer]
        if (buyer in assignment_dict):
            exclude_list = exclude_list + assignment_dict[buyer]
        
        recipient = get_random_name_excluding(recipient_list, exclude_list)

        ## Remove recipient from recipient list
        recipient_list.remove(recipient)

        ## Add buyer-> recipient pair
        new_assignment_dict[buyer] = [recipient]
    
    return new_assignment_dict

ass_dict1 = get_name_assignment(NAME_LIST, {})
print(ass_dict1)
ass_dict2 = get_name_assignment(NAME_LIST, {})
print(ass_dict2)
ass_dict3 = get_name_assignment(NAME_LIST, ass_dict1)
print(ass_dict3)


{'Tyler': ['Jenny'], 'Nancy': ['Peter'], 'Jenny': ['Nancy'], 'Sandy': ['Tyler'], 'Kent': ['Sandy'], 'Peter': ['Kent']}
